In [8]:
import numpy as np
import json,pickle,os,string, tqdm
from collections import defaultdict, Counter
from itertools import groupby
import kenlm
import re

In [9]:
with open("out.txt","rb") as f:
    out = pickle.load(f)[0]
with open("labels.json") as label_file:
    labels = str(''.join(json.load(label_file))) # "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "

In [10]:
def ctc_best_path(out,labels):
    "implements best path decoding as shown by Graves"
    out = [labels[i] for i in np.argmax(out, axis=1) if i!=labels[-1]]
    o = ""
    for i,j in groupby(out):
        o = o + i
    return o.replace("_","")

In [11]:
gred_txt = ctc_best_path(out,labels)
gred_txt

'AND AT FIRST THIS SORT OF THING IS UNPLEASANT ENOUGH TIT TOUCH ES ONE SENSE OF HONOR'

In [52]:
true_txt = 'AT FIRST THIS SORT OF THING IS UNPLEASANT ENOUGH IT TOUCHES ONE\'S SENSE OF HONOUR'

In [88]:
model = kenlm.LanguageModel('/home/hemant/deep/lm/libri_lm/3-gram.binary')

In [89]:
def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
#         w = [i[0] for i in ptot]
#         print(w)
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]
        return [i[1] for i in dict_]

In [101]:
def ctc_beam_search(out,labels, prune=0.0001, k=20, lm=None,alpha=0.3,beta=5):
    "implements CTC Prefix Search Decoding Algorithm as shown by Graves"

    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    out = np.vstack((np.zeros(F), out))
    steps = out.shape[0]
    W = lambda l: re.findall(r'\w+[\s|>]', l)
    
    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 1, 0
    prev_beams = ['']
    for t in range(1,steps):
        pruned_alphabet = [labels[i] for i in np.where(out[t] > prune)[0]]
        for b in prev_beams:
            for c_t in pruned_alphabet:
                index = labels.index(c_t)
                #Collapsing case (copy case as the last character in the beam)
                if c_t == "_": #Extending with a blank
                    pb[t][b] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])   
                else:
                    i_plus = b + c_t
                    if len(b) > 0 and c_t == b[-1]: #Extending with the same character as the last one
                        pnb[t][b] += out[t][index]*pnb[t-1][b]
                        pnb[t][i_plus] += out[t][index]*pb[t-1][b]
                    #expanding the beam (extend case as the last character is different)
                    elif c_t == " " and len(b.replace(' ', '')) > 0 : # LM constraints
                        prob = [i[0] for i in lm.full_scores(i_plus,eos=False,bos=False)][-1]
                        lm_p = (10**prob)**alpha
                        pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    else:
                        pnb[t][i_plus] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])

        ptot = pb[t] + pnb[t]
        for i in ptot.keys():
            ptot[i] = ptot[i]*(len(i)+1)**beta
        prev_beams = sort_beam(ptot,k)
    return prev_beams[0]

In [102]:
beam_txt=ctc_beam_search(out,labels,0.0001,k=1000,lm=model)

In [103]:
beam_txt

'AND AT FIRST THIS SORT OF THING IS UNPLEASANT ENOUGH DHIT TOUCHIS ONE SENSE OF HONOR'

In [104]:
print('true-----',true_txt)
print('gred-----',gred_txt)
print('beam-----',beam_txt)

true----- AT FIRST THIS SORT OF THING IS UNPLEASANT ENOUGH IT TOUCHES ONE'S SENSE OF HONOUR
gred----- AND AT FIRST THIS SORT OF THING IS UNPLEASANT ENOUGH TIT TOUCH ES ONE SENSE OF HONOR
beam----- AND AT FIRST THIS SORT OF THING IS UNPLEASANT ENOUGH DHIT TOUCHIS ONE SENSE OF HONOR


In [105]:
beam_out = 'AND AT FIRST THIS SORT OF THING IS UNPLEASANT ENOUGH DETACHES ONE SENSE OF HONOR'

In [106]:
gred_out = 'AND AT FIRST THIS SORT OF THING IS UNPLEASANT ENOUGH TIT TOUCH ES ONE SENSE OF HONOR'